In [11]:
from datasets import load_dataset

In [5]:
ds = load_dataset("allenai/c4", "realnewslike", split="train", streaming=True)

In [10]:
for b in ds:
    print(b["url"])
    break

https://www.catholic.org/encyclopedia/view.php?id=11981


In [12]:
ds = load_dataset("nhagar/c4_urls_realnewslike", streaming=True)

In [13]:
ds

IterableDatasetDict({
    train: IterableDataset({
        features: ['url', 'domain'],
        num_shards: 1
    })
})